<a href="https://colab.research.google.com/github/Momilijaz96/VGG16_CIFAR100_Optim/blob/main/VGG16_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Import CIFAR-10 and Resize

In [3]:
#from torchvision.transforms.transforms import RandomInvert
resize = transforms.Compose([transforms.Resize(64),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_set = datasets.CIFAR10(root="data", train=True, download=True, transform=resize)

In [5]:
test_set = datasets.CIFAR10(root="data", train=False, download=True, transform=resize)

Files already downloaded and verified


In [6]:
train_set[0][0].shape

torch.Size([3, 64, 64])

In [7]:
test_set[0][0].shape

torch.Size([3, 64, 64])

# VGG16

In [ ]:
import torch.nn as nn
import torchvision.models as models

model = models.vgg16(pretrained=True) #130million+ parameters

#Freeze all model parameters(convolution layers )
for param in model.features.parameters():
    param.requires_grad = False

#Change output classes
model.classifier[6].out_features = 10

In [9]:
model=model.to(device)

In [10]:
device

device(type='cuda')

In [11]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Training

In [12]:
train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=True)

In [13]:
criterion=torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr=0.001)

In [14]:
n_epochs = 10
train_acc_l = []
test_acc_l = []
for epoch in range(n_epochs):
  train_loss = []
  train_running_loss = 0
  train_acc = 0
  for data, targets in train_dataloader:
    data=data.to(device)
    targets=targets.to(device)
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    train_running_loss += loss
    #Get accuracy
    _, preds = torch.max(out.data, 1)
    train_acc += (preds == targets).sum()
    #Reset the gradients
    optimizer.zero_grad()
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()

  train_loss = train_running_loss/len(train_dataloader.dataset)
  train_acc_l.append(100 * train_acc/len(train_dataloader.dataset))

  #Evaluation
  model.eval()

  #Test Evaluation
  test_acc = 0
  for data, targets in test_dataloader:
    data = data.to(device)
    targets = targets.to(device)
    ps = model(data)
    #Get index of class label
    _,preds = torch.max(ps,1)
    #Get accuracy
    test_acc += torch.sum(preds == targets)
  test_acc_l.append(100* test_acc/len(test_dataloader.dataset))

  print(f'Epoch: {epoch+1}\t Train_Loss: {train_loss:.4f}\t Train_Acc: {train_acc_l[-1]:.4f}\t Test_Acc: {test_acc_l[-1]:.4f}')

  model.train()

Epoch: 1	 Train_Loss: 0.0187	 Train_Acc: 62.5780	 Test_Acc: 73.5800
Epoch: 2	 Train_Loss: 0.0159	 Train_Acc: 68.3260	 Test_Acc: 74.0600
Epoch: 3	 Train_Loss: 0.0148	 Train_Acc: 70.5380	 Test_Acc: 75.4200
Epoch: 4	 Train_Loss: 0.0140	 Train_Acc: 71.9880	 Test_Acc: 75.9800
Epoch: 5	 Train_Loss: 0.0131	 Train_Acc: 73.3260	 Test_Acc: 75.3600
Epoch: 6	 Train_Loss: 0.0128	 Train_Acc: 74.2360	 Test_Acc: 76.3100
Epoch: 7	 Train_Loss: 0.0124	 Train_Acc: 75.1760	 Test_Acc: 74.3200
Epoch: 8	 Train_Loss: 0.0119	 Train_Acc: 76.1000	 Test_Acc: 76.3200
Epoch: 9	 Train_Loss: 0.0114	 Train_Acc: 77.0140	 Test_Acc: 77.7800
Epoch: 10	 Train_Loss: 0.0110	 Train_Acc: 77.9760	 Test_Acc: 77.1900
